In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from qutip import *
from cqed_tools.calibration import *
from cqed_tools.mf import *
from copy import deepcopy
%matplotlib inline

/homes/pbrookes/anaconda2/envs/bistable/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [22]:
power_calibration = pd.read_csv('./exp_comparison/power_calibration.csv', header=None, index_col=0)
eps0 = power_calibration.T['eps0'].values[0]
params = qload('params')

Loaded instance object.


In [23]:
base_Ec = params.Ec
base_fc = params.fc
base_Ej = params.Ej
base_g = params.g
base_gamma_phi = params.gamma_phi
base_kappa_phi = params.kappa_phi
base_gamma = params.gamma
base_eps = 0.0
base_kappa = params.kappa
base_n_t = params.n_t
base_n_c = params.n_c
fd = 0.0

name = 'ID=15-2_high_power_twochi=15-1MHz_alpha=-0-21896_kappa=1-3MHz_nc=0-03_nt=0_eps0=2-41MHz'

power_list = np.array([-17])
sweep_list = eps_calc(power_list, eps0)
eps_list = sweep_list

endtime_list = [3e4 for param in sweep_list]
snapshots_list = [2001 for param in sweep_list]

In [25]:
def frequencies_gen(fd0, fd1, fd2, fd3, df0, df1, df2):
    
    frequencies1 = np.arange(fd0, fd1+df0, df0)
    frequencies2 = np.arange(fd1, fd2+df1, df1)
    frequencies3 = np.arange(fd3, fd2-df2, -df2)

    frequencies = np.hstack([frequencies1, frequencies2, frequencies3])
    
    frequencies = np.round(frequencies, 10)
    
    frequencies = np.array(sorted(set(list(frequencies))))
    
    return frequencies


def t_gen(eps_array):
    n_spectra = eps_array.shape[0]
    t_array = np.zeros(n_spectra, dtype=np.int)
    t_array += 3 * (eps_array <= 1e-5)
    t_array += 4 * (1e-5 < eps_array) * (eps_array <= 1e-4)
    t_array += 6 * (1e-4 < eps_array) * (eps_array <= 1e-3)
    t_array += 7 * (1e-3 < eps_array) * (eps_array <= 2e-3)
    t_array += 8 * (2e-3 < eps_array) * (eps_array <= 3e-3)
    t_array += 9 * (3e-3 < eps_array) * (eps_array <= 4e-3)
    t_array += 9 * (4e-3 < eps_array) * (eps_array <= 5e-3)
    t_array += 9 * (5e-3 < eps_array) * (eps_array <= 6e-3)
    t_array += 9 * (6e-3 < eps_array) * (eps_array <= 7e-3)
    t_array += 9 * (7e-3 < eps_array) * (eps_array <= 8e-3)
    t_array += 10 * (8e-3 < eps_array)
    return t_array - 2

def c_gen(eps_array):
    n_spectra = eps_array.shape[0]
    c_array = np.zeros(n_spectra, dtype=np.int)
    c_array += 3 * (eps_array <= 1e-5)
    c_array += 5 * (1e-5 < eps_array) * (eps_array <= 1e-4)
    c_array += 11 * (1e-4 < eps_array) * (eps_array <= 1e-3)
    c_array += 20 * (1e-3 < eps_array) * (eps_array <= 2e-3)
    c_array += 30 * (2e-3 < eps_array) * (eps_array <= 3.0e-3)
    c_array += 40 * (3e-3 < eps_array) * (eps_array <= 4e-3)
    c_array += 50 * (4e-3 < eps_array) * (eps_array <= 5e-3)
    c_array += 55 * (5e-3 < eps_array) * (eps_array <= 6e-3)
    c_array += 65 * (6e-3 < eps_array) * (eps_array <= 7e-3)
    c_array += 75 * (7e-3 < eps_array) * (eps_array <= 8e-3)
    c_array += 80 * (8e-3 < eps_array)
    return c_array - 5

In [26]:
fd1_list = [10.4722]
fd2_list = [10.4732]

fd_array = np.linspace(10.45, 10.49, 2001)

fd0_list = []
fd3_list = []

for eps in eps_list:
    params_instance = deepcopy(params)
    params_instance.eps = eps
    mf_amplitude_frame = mf_characterise(params_instance, fd_array)
    fd0_list.append(mf_amplitude_frame.dropna().index[0])
    fd3_list.append(mf_amplitude_frame.dropna().index[-1])

2001it [00:02, 733.84it/s]
2001it [00:02, 750.55it/s]


In [27]:
df0_list = [0.001 for param in sweep_list]
df1_list = [0.0002 for param in sweep_list]
df2_list = [0.001 for param in sweep_list]

In [28]:
group_folders = ['-17dBm']

In [29]:
gamma_phi_list = [base_gamma_phi for param in sweep_list]
gamma_list = [base_gamma for param in sweep_list]
nc_list = [base_n_c for param in sweep_list]
nt_list = [base_n_t for param in sweep_list]
fc_list = [base_fc for param in sweep_list]
kappa_list = [base_kappa for param in sweep_list]
g_list = [base_g for param in sweep_list]
Ec_list = [base_Ec for param in sweep_list]
Ej_list = [base_Ej for param in sweep_list]

eps_list = np.array(eps_list)
t_list = t_gen(eps_list)
c_list = c_gen(eps_list)

content = [eps_list, fd0_list, fd1_list, fd2_list, fd3_list, df0_list, df1_list, df2_list, t_list, c_list, endtime_list, snapshots_list, group_folders, gamma_list, nc_list,kappa_list,nt_list,g_list,gamma_phi_list,Ec_list,Ej_list,fc_list]

columns = ['eps', 'fd0', 'fd1', 'fd2', 'fd3', 'df0', 'df1', 'df2', 't_levels', 'c_levels', 'endtime', 'snapshots', 'group_folder','gamma', 'n_c','kappa','n_t','g','gamma_phi','Ec','Ej','fc']

recipe = pd.DataFrame(content).T
recipe.columns = columns

In [31]:
qubit_states = np.array([1])

#columns = ['eps','fd','qubit_state','t_levels','c_levels','fc','Ej','g','Ec','kappa', 'gamma', 'gamma_phi', 'n_t', 'n_c', 'end_time', 'snapshots', 'group_folder', 'completed', 'running']
columns = ['eps','fd','qubit_state','t_levels','c_levels','fc','Ej','g','Ec','kappa', 'gamma', 'gamma_phi', 'n_t', 'n_c', 'end_time', 'snapshots', 'group_folder']


queue_list = []

for index in range(recipe.shape[0]):
    row = recipe.iloc[index,:]
    frequencies = frequencies_gen(row.fd0, row.fd1, row.fd2, row.fd3, row.df0, row.df1, row.df2)

    #arrays = np.meshgrid(row.eps, frequencies, qubit_states, row.t_levels, row.c_levels, fc, Ej, g, Ec, kappa, gamma, gamma_phi, n_t, n_c, row.endtime, row.snapshots, 1, completed, running, indexing='ij')
    arrays = np.meshgrid(row.eps, frequencies, qubit_states, row.t_levels, row.c_levels, row.fc, row.Ej, row.g, row.Ec, row.kappa, row.gamma, row.gamma_phi, row.n_t, row.n_c, row.endtime, row.snapshots, row.group_folder, indexing='ij')
    #shape = arrays[16].shape
    #arrays[16] = np.tile(row.group_folder,shape)
    
    flattened = []
    for array in arrays:
        flattened.append(array.flatten())
    
    df = pd.DataFrame(flattened).T
    df.columns = columns
    
    queue_list.append(df)
    
combined_queue = pd.concat(queue_list)
combined_queue.index = np.arange(combined_queue.shape[0])
combined_queue.index.name = 'job_index'



with open('stack.csv','w') as f:
    f.write(name+'\n')
combined_queue.to_csv('stack.csv',mode='a')